In [1]:
using Random, Plots, Distributions, StatsBase, DataFrames, CSV, Plots.PlotMeasures

In [2]:
df = DataFrame(CSV.File("./mad1WTGFP/FISH_toShare_NoTSlabel_celldata_long_220620_simple_woMad3Outlier_withColDescription.csv"));

In [3]:
names(df)

15-element Vector{String}:
 "sampleID"
 "gene"
 "probe"
 "strain"
 "imageID"
 "CellNo"
 "NucleiCount"
 "Compartment"
 "CountMethod"
 "RNAperCell"
 "Cell_Pseudovol_um"
 "Cell_Area_um"
 "Cell_Length_um"
 "Cell_Width_um"
 "Nuc_Vol_um"

In [4]:
df = df[:,[:gene,:NucleiCount,:Compartment,:CountMethod,:RNAperCell,:Cell_Length_um]];

In [35]:
# filter NucleiCount = 1
dfa = subset(df,:NucleiCount => n -> n.==1)
# make cyto and nuc table separately.
dfcyto = subset(dfa, :Compartment=>c->c.=="Cytoplasm")
dfnuc = subset(dfa, :Compartment=>c->c.=="Nucleus")
# separate the 7 different genes
dfsN = []; dfsC = [];
genes = unique(dfa[!,:gene])
# add each to the vectors
for i in 1:10
    dftempN = subset(dfnuc, :gene=>S->S.==genes[i])
    dftempC = subset(dfcyto, :gene=>S->S.==genes[i])
    push!(dfsN,dftempN)
    push!(dfsC,dftempC)
end

In [36]:
n_vols = [dfsN[i][!,:Cell_Length_um] for i in 1:length(dfsN)]
c_vols = [dfsC[i][!,:Cell_Length_um] for i in 1:length(dfsN)]
n_ints = [dfsN[i][!,:RNAperCell] for i in 1:length(dfsN)]
c_ints = [dfsC[i][!,:RNAperCell] for i in 1:length(dfsN)];

In [37]:
Ns = [length(c_ints[i]) for i in 1:7];

For the linear model fits we use the Julia package GLM. This is a very convenient and fast way to construct linear models.

In [38]:
using GLM

Perform the linear model fits. Note in the below that our fitting parameters are stored in `npar` and `cpar` in the order `[intercept, gradient]`.

In [39]:
nlms = [lm(@formula(RNAperCell ~ Cell_Length_um), dfsN[i]) for i in 1:length(dfsN)]
npars = [coef(nlms[i]) for i in 1:length(dfsN)]
cs = hcat(npars...)[1,:]; ds = hcat(npars...)[2,:]
clms = [lm(@formula(RNAperCell ~ Cell_Length_um), dfsC[i]) for i in 1:length(dfsN)]
cpars = [coef(clms[i]) for i in 1:length(dfsN)]
as = hcat(cpars...)[1,:]; bs = hcat(cpars...)[2,:]

10-element Vector{Float64}:
  3.2543858520376427
  0.3027547144895288
  1.862936344101668
  0.4486839874995193
  0.4733008577390787
  0.3872917294645617
  0.5636688403707902
  0.33229267946068464
 -0.01903586284508915
  0.6386458986387803

A quick look at the linear model fit itself will tell us some properties of the fit.

In [40]:
clms[1]

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

RNAperCell ~ 1 + Cell_Length_um

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)     -12.2665    0.40463    -30.32    <1e-99  -13.0598   -11.4731
Cell_Length_um    3.25439   0.0423445   76.85    <1e-99    3.17136    3.33741
─────────────────────────────────────────────────────────────────────────────

Now, to construct the volume corrected moments we first need to know the mean values for the volumes and intensities. Collect these from the data.

In [41]:
n_vol_avgs = [mean(n_vols[i]) for i in 1:length(dfsN)]
c_vol_avgs = [mean(c_vols[i]) for i in 1:length(dfsN)]
n_avgs = [mean(n_ints[i]) for i in 1:length(dfsN)]
c_avgs = [mean(c_ints[i]) for i in 1:length(dfsN)]

10-element Vector{Float64}:
 18.287384616295455
  1.7151718625745296
 17.610967268107835
  2.818037297496365
  2.853951370509465
  4.341669328347927
  3.471709988317795
  5.38877324092077
 24.879556910694966
  3.2043133962584487

Note that even the means of the cytoplasmic can vary quite significantly between the replicates. As can the observed mean cell volumes in each experiment.

We know theoretically that to first order volume corrections to the means are nil. Let's just clarify this.

In [42]:
ds

10-element Vector{Float64}:
  0.2245899921869469
  0.023980893063671654
  0.07916482934427854
  0.03721835148155214
  0.05860329908418444
  0.014392613422049557
  0.025157783464598515
 -0.07699645804600715
 -0.42355363127884743
  0.07902465901807378

In [43]:
vc_mean_ns = [cs[i] + n_vol_avgs[i]*ds[i] for i in 1:length(dfsN)]
vc_mean_cs = [as[i] + c_vol_avgs[i]*bs[i] for i in 1:length(dfsN)]
# vc_mean_cs = a + c_vol_avg*b

10-element Vector{Float64}:
 18.287384616295448
  1.7151718625745302
 17.61096726810785
  2.8180372974963643
  2.853951370509466
  4.341669328347927
  3.4717099883177935
  5.388773240920772
 24.87955691069497
  3.2043133962584456

In [44]:
μ20s = [moment(n_ints[i], 2, n_avgs[i]) for i in 1:length(dfsN)];
μ02s = [moment(c_ints[i], 2, c_avgs[i]) for i in 1:length(dfsN)];
μ2Ωs = [moment(n_vols[i], 2, n_vol_avgs[i]) for i in 1:length(dfsN)];

In [45]:
n_ints[1];

In [46]:
μ11s = [(sum(n_ints[i].*c_ints[i])/length(n_ints[i])) - n_avgs[i]*c_avgs[i] for i in 1:length(dfsN)]

10-element Vector{Float64}:
  2.9335350934066327
 -0.0836773344948758
  0.48620505966952265
 -0.03015119208490591
 -0.18106725911585242
 -0.12170287246755862
 -0.028301554043778765
 -0.23404003525394756
 26.9683403826363
  1.2544683731175614

In [47]:
μ20s_i = [μ20s[i] - ds[i]^2 * μ2Ωs[i] for i in 1:length(dfsN)]
μ02s_i = [μ02s[i] - bs[i]^2 * μ2Ωs[i] for i in 1:length(dfsN)];

In [48]:
genes

10-element Vector{String31}:
 "cdc13endog"
 "mad1endog"
 "rpb1endog"
 "mad1WTymEGFP"
 "mad2WTymEGFP"
 "bub1WTymEGFP"
 "mad3WTymEGFP"
 "sep1WTymEGFP-hph"
 "SPAC27D709cWTymEGFP-hph"
 "SPAC2H1001WTymEGFP-hph"

In [49]:
μ11s_i = [μ11s[i] - ds[i]*bs[i] * μ2Ωs[i] for i in 1:length(dfsN)]

10-element Vector{Float64}:
  0.6191902799315514
 -0.10095622435350413
  0.04742611860024876
 -0.08693640723020321
 -0.259091630280019
 -0.14074223584828016
 -0.08221160809740848
 -0.1619172598908808
 26.951994588562876
  1.1327239805867395

Clearly the volume corrected means are simply the means if one averaged over the data. Can now store the values for the VC FF's along with the mean cyto #.

In [50]:
nuc_FFs = μ20s_i./vc_mean_ns

10-element Vector{Float64}:
 0.9159105819018832
 0.919476115404893
 0.9490127578051145
 0.983497126450783
 0.8537487742762156
 0.9475595569623816
 0.9655228330073937
 0.8475446192260743
 2.6213179658115164
 1.9368249625358231

In [51]:
cyto_FFs = μ02s_i./vc_mean_cs

10-element Vector{Float64}:
 1.0648919795756426
 0.627576686821267
 0.7623392272382672
 0.6725957863020816
 0.8431401698178076
 0.646448735849169
 0.7239662326468429
 0.9175703591365221
 9.943713931875054
 4.103832374474887

In [52]:
cyto_means = vc_mean_cs

10-element Vector{Float64}:
 18.287384616295448
  1.7151718625745302
 17.61096726810785
  2.8180372974963643
  2.853951370509466
  4.341669328347927
  3.4717099883177935
  5.388773240920772
 24.87955691069497
  3.2043133962584456